## Calcular Métricas - Tempo de Votação, Biometria, etc.

---

## Importing libraries

In [1]:
import duckdb
import pandas as pd
import time

## Importing Data

In [2]:
cursor = duckdb.connect()

In [3]:
TABLE = "read_parquet('VOTES.parquet/*/*/*/*.parquet', hive_partitioning=True)"

In [13]:
source = F"""
    (
    SELECT 
        *,
        CASE event_date
            WHEN '2022-10-02' THEN 1
            WHEN '2022-10-03' THEN 1
            WHEN '2022-10-30' THEN 2
            WHEN '2022-10-31' THEN 2
            ELSE NULL
        END::INT AS turno,
        COALESCE(
            timestamp_biometria_1,
            timestamp_biometria_2,
            timestamp_biometria_3,
            timestamp_biometria_4,
            timestamp_biometria_manual
        ) AS timestamp_biometria_final
    FROM 
        {TABLE}
    WHERE uf IN ('RN')
    ) _
"""

In [14]:
source
df = cursor.execute(
    F"""
    SELECT 
        *
    FROM 
        {source}
    LIMIT 10
"""
).df()
df.columns
df

,event_date,uf,filename,vote_id,city_code,zone_code,zone_group,section_code,quantidade_votos_computados,quantidade_solicitacoes_biometria,...,timestamp_biometria_1,timestamp_biometria_2,timestamp_biometria_3,timestamp_biometria_4,timestamp_biometria_manual,timestamp_habilitacao_eleitor,timestamp_titulo_digitado,timestamp_voto_computado,turno,timestamp_biometria_final
0,2022-10-02,RN,o00407-1600400210044,53.0,16004,0021,0-100,0044,1.0,3.0,...,2022-10-02 10:01:52,2022-10-02 10:02:16,2022-10-02 10:02:29,NaT,NaT,2022-10-02 10:02:36,2022-10-02 10:01:49,2022-10-02 10:04:38,1,2022-10-02 10:01:52
1,2022-10-02,RN,o00407-1600400210044,113.0,16004,0021,0-100,0044,1.0,1.0,...,2022-10-02 12:00:43,NaT,NaT,NaT,NaT,2022-10-02 12:00:51,2022-10-02 12:00:41,2022-10-02 12:01:58,1,2022-10-02 12:00:43
2,2022-10-02,RN,o00407-1600400210044,151.0,16004,0021,0-100,0044,1.0,3.0,...,2022-10-02 13:13:12,2022-10-02 13:13:24,2022-10-02 13:13:34,NaT,NaT,2022-10-02 13:13:50,2022-10-02 13:13:08,2022-10-02 13:14:46,1,2022-10-02 13:13:12
3,2022-10-02,RN,o00407-1600400210044,176.0,16004,0021,0-100,0044,1.0,1.0,...,2022-10-02 14:05:31,NaT,NaT,NaT,NaT,2022-10-02 14:05:44,2022-10-02 14:05:28,2022-10-02 14:06:31,1,2022-10-02 14:05:31
4,2022-10-02,RN,o00407-1600400210044,188.0,16004,0021,0-100,0044,1.0,5.0,...,2022-10-02 14:30:43,2022-10-02 14:30:52,2022-10-02 14:31:00,2022-10-02 14:31:09,2022-10-02 14:31:18,2022-10-02 14:31:48,2022-10-02 14:30:41,2022-10-02 14:32:36,1,2022-10-02 14:30:43
5,2022-10-02,RN,o00407-1600400210044,271.0,16004,0021,0-100,0044,1.0,2.0,...,2022-10-02 17:17:19,2022-10-02 17:17:27,NaT,NaT,NaT,2022-10-02 17:17:33,2022-10-02 17:17:18,2022-10-02 17:18:31,1,2022-10-02 17:17:19
6,2022-10-02,RN,o00407-1600400210047,25.0,16004,0021,0-100,0047,1.0,5.0,...,2022-10-02 09:02:36,2022-10-02 09:02:45,2022-10-02 09:02:52,2022-10-02 09:03:05,2022-10-02 09:03:14,2022-10-02 09:03:46,2022-10-02 09:02:33,2022-10-02 09:05:35,1,2022-10-02 09:02:36
7,2022-10-02,RN,o00407-1600400210047,95.0,16004,0021,0-100,0047,1.0,1.0,...,2022-10-02 11:45:13,NaT,NaT,NaT,NaT,2022-10-02 11:45:22,2022-10-02 11:45:12,2022-10-02 11:46:07,1,2022-10-02 11:45:13
8,2022-10-02,RN,o00407-1600400210047,100.0,16004,0021,0-100,0047,0.0,5.0,...,2022-10-02 11:55:33,2022-10-02 11:55:56,2022-10-02 11:56:04,2022-10-02 11:56:22,2022-10-02 11:56:30,NaT,2022-10-02 11:55:31,NaT,1,2022-10-02 11:55:33
9,2022-10-02,RN,o00407-1600400210047,123.0,16004,0021,0-100,0047,1.0,2.0,...,2022-10-02 12:43:11,2022-10-02 12:43:23,NaT,NaT,NaT,2022-10-02 12:43:30,2022-10-02 12:43:08,2022-10-02 12:44:31,1,2022-10-02 12:43:11


## Preparinga Data

Méticas no Cubo OLAP - Turno, UF, Zona, Seção.

- Número de Votos
- Número de Seções Eleitorais
- Média, Soma, q50%, q90% do Tempo total de Voto, Tempo de Biometria, Tempo Total

- Quantidade de Votos efetuados em até 30s, 1min, 1min30s, 2min, 2min30s, 3min+
- Taxa de Sucesso da Biometria em 1 tentativa, 2 tentativas, 3 tentativas, 4 tentativas, Falha
- Quantidade de Teclas Pressionadas
- Quantidade de Cargos Distintos Votados

In [15]:
tempo_voto_total = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_titulo_digitado))"
tempo_voto       = "EXTRACT(EPOCH FROM (timestamp_voto_computado  - timestamp_habilitacao_eleitor))"
tempo_biometria  = "EXTRACT(EPOCH FROM (timestamp_biometria_final - timestamp_titulo_digitado))"

intervalos_tempo_segundos_votos = [0, 30, 60, 90, 120, 150, 180, 210, 300, 9999]
condicionais_intervalo_tempo_segundos_votos = ", ".join([
    F"""
    SUM( 
        CASE WHEN 
        {tempo_voto} >= {intervalos_tempo_segundos_votos[i]} 
        AND {tempo_voto} < {intervalos_tempo_segundos_votos[i+1]}
        THEN 1 ELSE 0 END 
    ) AS votos_{intervalos_tempo_segundos_votos[i]}_{intervalos_tempo_segundos_votos[i+1]}_segundos
    """
    for i in range(0, len(intervalos_tempo_segundos_votos)-1)
])

In [16]:


query = F"""
    SELECT
        turno,
        uf, 
        zone_code, 
        section_code,

        COUNT(*) AS total_votos,
        COUNT( DISTINCT section_code ) AS total_secoes,

        SUM( {tempo_voto} ) AS tempo_voto_soma,
        AVG( {tempo_voto} ) AS tempo_voto_medio,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY {tempo_voto}) AS tempo_voto_mediana,
        PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY {tempo_voto}) AS tempo_voto_90percentil,

        SUM( {tempo_biometria} ) AS tempo_biometria_soma,
        AVG( {tempo_biometria} ) AS tempo_biometria_medio,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY {tempo_biometria}) AS tempo_biometria_mediana,
        PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY {tempo_biometria}) AS tempo_biometria_90percentil,

        SUM( {tempo_voto_total} ) AS tempo_voto_total_soma,
        AVG( {tempo_voto_total} ) AS tempo_voto_total_medio,
        PERCENTILE_CONT(0.5) WITHIN GROUP(ORDER BY {tempo_voto_total}) AS tempo_voto_total_mediana,
        PERCENTILE_CONT(0.9) WITHIN GROUP(ORDER BY {tempo_voto_total}) AS tempo_voto_total_90percentil,
        
        {condicionais_intervalo_tempo_segundos_votos},
        1-AVG(biometria_nao_funcionou::INT) AS tx_sucesso_biometria,

    FROM
        {source}
    WHERE quantidade_votos_computados = 1
    GROUP BY ROLLUP(turno, uf, zone_code, section_code)
"""

df = cursor.execute(query+" LIMIT 10").df()
df.columns
df

,turno,uf,zone_code,section_code,total_votos,total_secoes,tempo_voto_soma,tempo_voto_medio,tempo_voto_mediana,tempo_voto_90percentil,...,votos_0_30_segundos,votos_30_60_segundos,votos_60_90_segundos,votos_90_120_segundos,votos_120_150_segundos,votos_150_180_segundos,votos_180_210_segundos,votos_210_300_segundos,votos_300_9999_segundos,tx_sucesso
0,2.0,None,None,None,2104211,628,27711090.0,13.169349,11.0,20.0,...,2024569.0,66559.0,8858.0,2284.0,839.0,392.0,230.0,305.0,175.0,0.888645
1,NaN,None,None,None,4141015,628,161448488.0,38.987661,29.0,79.0,...,2093496.0,1274122.0,457256.0,146467.0,68740.0,37245.0,21683.0,27542.0,14464.0,0.887552
2,1.0,None,None,None,2036804,626,133737398.0,65.660416,52.0,109.0,...,68927.0,1207563.0,448398.0,144183.0,67901.0,36853.0,21453.0,27237.0,14289.0,0.886423
3,1.0,RN,0037,None,18372,72,1339938.0,72.933703,56.0,126.0,...,193.0,10032.0,4551.0,1555.0,799.0,438.0,272.0,344.0,188.0,0.895656
4,1.0,RN,0053,None,25642,97,1767397.0,68.925864,53.0,116.0,...,802.0,14632.0,5813.0,1966.0,922.0,533.0,320.0,400.0,254.0,0.926488
5,1.0,RN,0005,None,54701,210,3601668.0,65.842818,52.0,110.0,...,1682.0,32320.0,12219.0,3881.0,1914.0,1045.0,593.0,712.0,335.0,0.933310
6,1.0,RN,0033,None,75630,270,4793173.0,63.376610,51.0,103.0,...,1649.0,46838.0,16781.0,5111.0,2269.0,1186.0,665.0,794.0,337.0,0.793852
7,1.0,RN,0045,None,18954,76,1377260.0,72.663290,56.0,122.0,...,223.0,10482.0,4672.0,1569.0,797.0,396.0,249.0,357.0,209.0,0.873641
8,2.0,RN,0068,None,19273,77,234869.0,12.186427,10.0,18.0,...,18672.0,487.0,84.0,17.0,4.0,4.0,1.0,2.0,2.0,0.943704
9,2.0,RN,0002,None,99746,333,1126442.0,11.293104,10.0,17.0,...,97598.0,1822.0,219.0,57.0,23.0,14.0,3.0,6.0,4.0,0.948399
